In [ ]:
# default_exp face_class_ver

In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/ubuntu/hw2p2'

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#export
# imports
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
from torch.nn.functional import cosine_similarity, adaptive_avg_pool2d
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
import pandas as pd
from functools import partial
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import json
from datetime import datetime
import os
import time

# from datasets import FaceClassificationDataset, FaceVerificationDataset
# from losses import CenterLoss
# from models.mobilenet import *
# from models.resnet import *
# from hyperoptimization import fit_predict
from hw2p2.datasets import FaceClassificationDataset, FaceVerificationDataset
from hw2p2.losses import CenterLoss
from hw2p2.models.mobilenet import *
from hw2p2.models.resnet import *
from hw2p2.hyperoptimization import fit_predict

In [15]:
#export
def main(model, sample_size=None, max_evals=20):

    # Hyperparameters space
    space = {'model': hp.choice(label='model', options=[model]),
             'in_channels': hp.choice(label='in_channels', options=[3]),
            #  'n_classes': hp.choice(label='n_classes', options=[4000]),
             'batch_size': scope.int(hp.choice(label='batch_size', options=[128,256,512])),
             'lr': hp.loguniform(label='lr', low=np.log(5e-3), high=np.log(5e-2)),
             'lr_decay': hp.choice(label='lr_decay', options=[0.5]),
             'n_lr_decay_steps': hp.choice(label='n_lr_decay_steps', options=[3]),
             'center_loss': hp.choice(label='center_loss', options=[True]),
             'lr_cl': hp.loguniform(label='lr_cl', low=np.log(5e-3), high=np.log(2e-1)),
             'alpha_cl': hp.choice(label='alpha_cl', options=[0.01,0.1,1]),
             'n_epochs': hp.choice(label='n_epochs', options=[10000]),
             'eval_steps': scope.int(hp.choice(label='eval_steps', options=[4])),}

    if sample_size==None:
        space['n_classes'] = hp.choice(label='n_classes', options=[4000])
    else:
        space['n_classes'] = hp.choice(label='n_classes', options=[sample_size])

    # Hyperparameters search
    trials = Trials()
    fmin_objective = partial(fit_predict, trials=trials, verbose=True, sample_size=sample_size)
    best = fmin(fmin_objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    
    keys = ['loss','val_c_loss','train_c_acc','val_c_acc','val_v_acc','run_time','trajectories']
    
    best_class = trials.results[np.argmax([r['val_c_acc'] for r in trials.results])]  
    best_class_res_dict = {key: best_class[key] for key in keys}
    best_class_model = best_class['model']
    best_class_mc = best_class['mc']
    best_class_time_stamp = best_class['time_stamp']
    
    s = 'hw2p2' + '_' + best_class_time_stamp
    filename = f'./results/{s}.pth'
    
    json_mc = json.dumps(best_class_mc)
    with open(f'./results/mc_{best_class_time_stamp}.json', 'w') as bfm: 
        bfm.write(json_mc)  
    json_res = json.dumps(best_class_res_dict)
    with open(f'./results/res_dict_{best_class_time_stamp}.json', 'w') as bfm: 
        bfm.write(json_res)  
        
    torch.save(best_class_model.model.state_dict(), filename)

    best_ver = trials.results[np.argmax([r['val_v_acc'] for r in trials.results])]
    best_ver_res_dict = {key: best_ver[key] for key in keys}
    best_ver_model = best_ver['model']
    best_ver_mc = best_ver['mc']
    best_ver_time_stamp = best_ver['time_stamp']
    
    s = 'hw2p2' + '_' + best_ver_time_stamp
    filename = f'./results/{s}.pth'
    
    json_mc = json.dumps(best_ver_mc)
    with open(f'./results/mc_{best_ver_time_stamp}.json', 'w') as bfm: 
        bfm.write(json_mc) 
    json_res = json.dumps(best_ver_res_dict)
    with open(f'./results/res_dict_{best_ver_time_stamp}.json', 'w') as bfm: 
        bfm.write(json_res)  
        
    torch.save(best_ver_model.model.state_dict(), filename)


In [16]:
os.getcwd()
# os.chdir('..')

'/home/ubuntu/hw2p2'

In [18]:
#export
if __name__ == "__main__": 
    main(model='resnet18', sample_size=None, max_evals=20)

                                                      
Current directory: /home/ubuntu/hw2p2

17-03-21_23-33-23                                     
alpha_cl                 0.1                          
batch_size               128
center_loss             True
eval_steps                 4
in_channels                3
lr                  0.005104
lr_cl               0.037255
lr_decay                 0.5
model               resnet18
n_classes               4000
n_epochs               10000
n_lr_decay_steps           3
dtype: object

train_dataset_len: 380638, val_c_dataset_len: 8000, val_v_dataset_len: 8805
==============================Start Fitting==============================
  0%|          | 0/20 [00:05<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [7]:
os.getcwd()

'/home/ubuntu/hw2p2'

In [10]:
import json

model_mc_file = './results/mc_17-03-21_23-28-42.json'
with open(model_mc_file) as f: mc = json.load(f)
model_mc_file = './results/res_dict_17-03-21_23-28-42.json'
with open(model_mc_file) as f: res = json.load(f)
model_file = './results/hw2p2_17-03-21_23-28-42.pth'

model = ResNetN(18,
                in_channels=mc['in_channels'],
                n_classes=mc['n_classes'],
                lr=mc['lr'],
                lr_decay=mc['lr_decay'],
                n_lr_decay_steps=mc['n_lr_decay_steps'],
                center_loss = mc['center_loss'],
                lr_cl=mc['lr_cl'],
                alpha_cl=mc['alpha_cl'],
                n_epochs=mc['n_epochs'],
                eval_steps=mc['eval_steps'])

model.model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [12]:
print(res)
print(mc)

{'loss': -1.0, 'val_c_loss': -1, 'train_c_acc': 0.05108173076923077, 'val_c_acc': 0.0625, 'val_v_acc': 0.6349206349206349, 'run_time': 14.432716846466064, 'trajectories': {'epoch': [0, 4], 'train_loss': [670.2711897629958, 372.0813950758714], 'train_c_acc': [0.05108173076923077, 0.046875], 'val_c_loss': [912.7404174804688, 917.6589050292969], 'val_c_acc': [0.0625, 0.0625], 'val_v_acc': [0.6349206349206349, 0.6349206349206349]}}
{'alpha_cl': 1, 'batch_size': 16, 'center_loss': True, 'eval_steps': 4, 'in_channels': 3, 'lr': 0.015801270540051794, 'lr_cl': 0.017213453712546033, 'lr_decay': 0.5, 'model': 'resnet18', 'n_classes': 20, 'n_epochs': 5, 'n_lr_decay_steps': 3}


In [13]:
model_mc_file = './results/mc_17-03-21_23-28-57.json'
with open(model_mc_file) as f: mc = json.load(f)
model_mc_file = './results/res_dict_17-03-21_23-28-57.json'
with open(model_mc_file) as f: res = json.load(f)
model_file = './results/hw2p2_17-03-21_23-28-57.pth'

model = ResNetN(18,
                in_channels=mc['in_channels'],
                n_classes=mc['n_classes'],
                lr=mc['lr'],
                lr_decay=mc['lr_decay'],
                n_lr_decay_steps=mc['n_lr_decay_steps'],
                center_loss = mc['center_loss'],
                lr_cl=mc['lr_cl'],
                alpha_cl=mc['alpha_cl'],
                n_epochs=mc['n_epochs'],
                eval_steps=mc['eval_steps'])

model.model.load_state_dict(torch.load(model_file))

<All keys matched successfully>

In [14]:
print(res)
print(mc)

{'loss': -1.0, 'val_c_loss': -1, 'train_c_acc': 0.10276442307692307, 'val_c_acc': 0.125, 'val_v_acc': 0.47619047619047616, 'run_time': 14.389012336730957, 'trajectories': {'epoch': [0, 4], 'train_loss': [7.035186199041513, 5.624412055198963], 'train_c_acc': [0.09254807692307693, 0.10276442307692307], 'val_c_loss': [12.188202381134033, 12.305235862731934], 'val_c_acc': [0.125, 0.125], 'val_v_acc': [0.47619047619047616, 0.47619047619047616]}}
{'alpha_cl': 0.01, 'batch_size': 16, 'center_loss': True, 'eval_steps': 4, 'in_channels': 3, 'lr': 0.029865896744795523, 'lr_cl': 0.15374739504606688, 'lr_decay': 0.5, 'model': 'resnet18', 'n_classes': 20, 'n_epochs': 5, 'n_lr_decay_steps': 3}
